## Hello everybody 😃 😃 welcome back


In [ ]:
import os
import cv2 
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Image Size
IMG_ROWS, IMG_COLS = 100, 100 # input image dimensions
NB_CLASSES =  3 # number of outputs = number of digits

## (01)   ACQUISITION DES DONNEES - IMAGES DES FEUILLES DES PLANTES. 

In [ ]:
########################################### function for plotting images

def plot_images(images, total_images=100, rows=20, cols=5, fsize=(20,100), titre='Image'):
    
    fig = plt.figure(figsize=fsize) # create a new figure window
    
    for i in range(total_images): # display images
        # subplot : 33 rows and 5 columns
        img_grid = fig.add_subplot(rows, cols, i+1)
        # plot features as image
        img_grid.imshow(images[i])
        
        plt.title(titre)


In [ ]:
############################################ function for resizng images
       
def preprocess_image(image, image_height=IMG_ROWS, image_width=IMG_COLS):

    return cv2.resize(image, (image_height, image_width))

############################################ function for reading images 
       
def read_images (path , sz= None ):
    
    print('\nCHARGEMENT DES IMAGES DE LA BASE .......................!\n') 

    X,y = [], []
    
    for dirname , dirnames , filenames in os.walk(path):
        
        c = 0
        
        for subdirname in dirnames :
            
            subject_path = os. path . join ( dirname , subdirname )
            
            for filename in os. listdir ( subject_path ):
                
                im = Image.open(os.path.join(subject_path, filename))
                #im = im.convert ("L")

                if (sz is not None ):
                    im = im.resize (sz , Image.ANTIALIAS ) 
                    
                im = np.array(im)
                im = preprocess_image(im, IMG_ROWS, IMG_COLS)
                X.append(im)
                y.append (c)  
                
            c = c+1
            
    return [X,y]


In [ ]:
############################################ Read And Resize test images

print('\n\nLECTURE DES IMAGES DE LA BASE D\'APPRENTISSAGE........!') 

[X_train, y_train] = read_images("C:/Users/Dell 7280/Documents/Etude/ESCEP/S3/Deep learning/Corrections/TP3-Patato/Data/Potato/TrainData") # Potato

print('\nAFFICHAGE DE QUELQUES IMAGES DE LA BASE.................!')
plot_images(X_train, 2, 1, 2,(10, 50), titre='Base D\'Apprentissage')
plt.show()
print('\nFIN D\'AFFICHAGE DES IMAGES DE LA BASE...................!\n')

In [ ]:
############################################################# Train Data

images_train = X_train
images_train = np.asarray(images_train)

# Train targets
train_features = images_train
train_targets = y_train 

from keras.utils import to_categorical 

# convert class vectors to binary class matrices
train_targets = to_categorical(train_targets, NB_CLASSES)

print('\nNORMALISATION DES BASES DE TEST ET D\'APPRENTISSAGE.\n')

########################################################## Normalisation

train_features = train_features.astype('float32')

mean_vals = np.mean(train_features, axis=0)
std_val = np.std(train_features)
train_features = (train_features - mean_vals)/std_val

train_features = train_features.reshape(train_features.shape[0], IMG_ROWS, IMG_COLS, 3)
print("train_features.shape     >==============<> : {}".format(train_features.shape))
print("train_targets.shape      >==============<> : {}".format(train_targets.shape))

In [ ]:
########################################################################
#(02)        CLASSIFICATION : CREATION DU MODELE DE PREDICTION         #
########################################################################

In [ ]:
from keras import optimizers
from time import time
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
######################################################## Build The Model

class AlexNet:
    
	@staticmethod
	def build(input_shape, classes): 
        
		model = Sequential() 
        
        # Add the 1st convolution layer (filters = 96, kernel size=11*11, stride=4)
		model.add(Conv2D(filters = 96, kernel_size=(11,11) , strides=(4, 4), padding="same", input_shape=input_shape))
		model.add(Activation("relu"))
		# Max pooling
		model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same"))

        # Add the 2nd convolution layer (filters = 256, kernel size=5*5, stride=1)
		model.add(Conv2D(filters = 96, kernel_size=(11,11) , strides=(4, 4), padding="same"))
		model.add(Activation("relu"))
		# Max pooling
		model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same"))

        # Add the 3rd convolution layer (filters = 384, kernel size=3*3, stride=1)
		model.add(Conv2D(filters = 384, kernel_size=(3,3), strides=(3, 3), padding="same"))
		model.add(Activation("relu"))
		
		# Add the 4th convolution layer (filters = 384, kernel size=3*3, stride=1)
		model.add(Conv2D(filters = 384, kernel_size=(3,3), strides=(4, 4), padding="same"))
		model.add(Activation("relu"))
		
		# Add the 5th convolution layer (filters = 256, kernel size=3*3, stride=1)
		model.add(Conv2D(filters = 256, kernel_size=(11,11), strides=(1, 1), padding="same"))
		model.add(Activation("relu"))
		# Max pooling
		model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same"))
		
        # Passing it to a fully Connected Layer FC, Here we do flatten!
		model.add(Flatten())
		
        # 1st Fully Connected Layer has 4096 neurons
		model.add(Dense(4096, input_shape=input_shape))
		model.add(Activation("relu"))
        
        # Add Dropout to prevent overfitting
		model.add(Dropout(0.4))
        
        # 2nd Fully Connected Layer has 4096 neurons
		model.add(Dense(4096))
		model.add(Activation("relu"))
        
        # Add Dropout
		model.add(Dropout(0.4))
        
        # Add a tanh activation function
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		return model

In [ ]:
########################################################## Use The Model

# Définition de l'optimiseur pour la compilation du modèle
OPTIMIZER = optimizers.Adam()

# Définition de la forme d'entrée des images pour le modèle (hauteur, largeur, nombre de canaux de couleur)
INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 3)

# Définition du nombre d'époques (cycles d'entraînement complets sur le dataset)
NB_EPOCH = 10

# Définition de la taille du lot (nombre d'échantillons par mise à jour de gradient)
BATCH_SIZE = 64

# Définition du niveau de verbosité (affichage des informations durant l'entraînement)
VERBOSE = 1

# Initialisation du modèle en utilisant l'architecture AlexNet avec la forme d'entrée et le nombre de classes
model = AlexNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

# Compilation du modèle avec l'optimiseur 'adam', la fonction de perte 'categorical_crossentropy', et l'évaluation de la précision
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

# Affichage d'un résumé de la structure du modèle
model.summary()

# Enregistrement du temps de début de l'entraînement
t_start = time()

# Entraînement du modèle avec les données d'entraînement (caractéristiques et cibles), en utilisant la taille de lot et le nombre d'époques définis
history = model.fit(train_features, train_targets, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE)

# Calcul du temps total d'entraînement
time_full_train = time() - t_start

# Affichage du temps d'entraînement du classificateur
print("\nTEMPS D'APPRENTISSAGE DU CLASSIFIEUR >====<> : %0.2fs " % (time_full_train))

# Sauvegarde du modèle entraîné dans un fichier
model.save("AlexNet_groupe_2.h5")

# Affichage d'un message indiquant que le modèle a été enregistré
print('\nENREGISTRER LE MODELE .\n')


# Measure of AlexNet for one dataset patato
1.  95.02%
2.  95.26%
3.  96.95%
4.  96.88%
5.  96.88%
6.  95.42%
7.  97.26%
8.  96.42%
9.  96.79%
10. 96.49%


Result of AlexNet for ten test = 96.43 %